In [72]:
import numpy as np
import pandas as pd
import healpy as hp
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from astropy.coordinates import SkyCoord
from astropy import units as u
from DS_Planck_Unet import train_pix_act
from DS_healpix_fragmentation import radec2pix, pix2pix, pix2radec, \
    one_pixel_fragmentation

In [2]:
pz = pd.read_csv('/home/rt2122/Data/clusters/planck_z.csv')
pnz = pd.read_csv('/home/rt2122/Data/clusters/planck_no_z.csv')
act = pd.read_csv('/home/rt2122/Data/clusters/act.csv')
true_df = pd.concat([pz, pnz, act], ignore_index=True)

In [3]:
true_df.shape

(5373, 54)

In [5]:
train_pix_act

[2, 8, 10, 11, 13, 15, 32, 33, 36, 37, 40, 42, 43, 44, 45, 46]

In [7]:
def get_small_pix(big_pix):
    from DS_healpix_fragmentation import one_pixel_fragmentation
    
    matr = one_pixel_fragmentation(2, big_pix, 10)
    return matr.flatten()

In [9]:
true_df['pix2'] = radec2pix(true_df['RA'], true_df['DEC'], 2)

In [14]:
sc = SkyCoord(ra=true_df['RA']*u.degree, dec=true_df['DEC']*u.degree, frame='icrs')
true_df['l'] = sc.galactic.l.degree
true_df['b'] = sc.galactic.b.degree

In [35]:
pix_dict = {}
for pix in tqdm(train_pix_act):
    true_cur = true_df[true_df['pix2'] == pix]
    vecs = hp.ang2vec(true_cur['l'], true_cur['b'], lonlat=True)
    train_space = set(get_small_pix(pix))
    for vec in vecs:
        cur_pix = hp.query_disc(2048, vec, radius=np.radians(15/60))
        train_space -= set(cur_pix)
    pix_dict[pix] = train_space

In [98]:
all_pix = []
for pix in pix_dict:
    all_pix.extend(list(pix_dict[pix]))

In [99]:
len(all_pix)

16751536

In [38]:
np.save('/home/rt2122/Data/class/fp_coords/fp_pz_pnz_act', np.array(all_pix))

In [39]:
for pix in pix_dict:
    np.save('/home/rt2122/Data/class/fp_coords/fp_{pix}'.format(pix=pix), 
            np.array(pix_dict[pix]))

In [101]:
df = pd.DataFrame({'pix' : np.array(all_pix), 'pix2' : [-1] * len(all_pix)})
df['pix2'] = pix2pix(np.array(list(all_pix)), 2048, 2)

In [107]:
df.to_csv('/home/rt2122/Data/class/fp_coords/fp_pz_pnz_act.csv')

In [102]:
len(df)

16750512

In [104]:
df = df.iloc[range(0, len(df), 2)]
len(df)

4187628

In [105]:
list(df)
df.index = np.arange(len(df))

In [106]:
print(type(np.array(df['pix'])))
ra, dec = pix2radec(list(df['pix']), nside=2048)
df['RA'] = ra
df['DEC'] = dec

<class 'numpy.ndarray'>


In [60]:
def draw_pic_class(ra, dec, radius=7.5/60, p_nside=2048):
    from DS_healpix_fragmentation import find_biggest_pixel, one_pixel_fragmentation,\
        draw_proper_circle, matr2dict
    
    o_nside, o_pix = find_biggest_pixel(ra, dec, radius)
    matr = one_pixel_fragmentation(o_nside, o_pix, int(np.log2(p_nside // o_nside)))
    return len(draw_proper_circle(ra, dec, radius, p_nside, mdict=matr2dict(matr), 
            shape=matr.shape, coords_mode=True))

In [108]:
min_len = 100
for i in tqdm(range(len(pz))):
    min_len = min(min_len, draw_pic_class(pz['RA'].iloc[i], pz['DEC'].iloc[i]))
min_len

17

In [109]:
min_len = 100
for i in tqdm(range(len(pnz))):
    min_len = min(min_len, draw_pic_class(pnz['RA'].iloc[i], pnz['DEC'].iloc[i]))
min_len

40

In [110]:
min_len = 100
for i in tqdm(range(len(act))):
    min_len = min(min_len, draw_pic_class(act['RA'].iloc[i], act['DEC'].iloc[i]))
min_len

KeyboardInterrupt: 

In [70]:
print([12,4, 2].extend([2, 3]))

None


In [100]:
#cut borders of pix2
pred_len = len(all_pix)
for pix2 in tqdm(train_pix_act):
    matr = one_pixel_fragmentation(2, pix2, 10)
    idx = list(range(32))
    idx.extend(list(range(1024-32, 1024)))
    all_pix = list(set(all_pix) - set(matr[idx, idx].flatten()))
print(len(all_pix) - pred_len)


-1024


In [76]:
len(all_pix)

16750512